<a href="https://colab.research.google.com/github/saurabh533/Lets_Grow_More/blob/main/Fashion_mnst_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
import keras

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [ ]:
#SCALING OUR IMAGES
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [ ]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
test_images.shape

(10000, 28, 28, 1)

In [ ]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(28, 28, 1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu',
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='my_project'
)


<ipython-input-28-d0d7f25ef293>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
tuner.search(train_images,train_labels, epochs=10, validation_split=0.1)


Trial 5 Complete [00h 02m 23s]
val_accuracy: 0.8774999976158142

Best val_accuracy So Far: 0.887333333492279
Total elapsed time: 00h 09m 24s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/my_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 112
learning_rate: 0.01
Score: 0.887333333492279

Trial 3 summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 5
dense_1_units: 48
learning_rate: 0.01
Score: 0.8809999823570251

Trial 1 summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 96
learning_rate: 0.01
Score: 0.8794999718666077

Trial 4 summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 64
learning_rate: 0.01
Score: 0.8774999976158142

Trial 2 summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 48
learning_rate: 0.01
Score: 0.1054999977350235


In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        27712     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 112)               4128880   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 4,158,202
Trainable params: 4,158,202
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model.fit(train_images,train_labels,epochs=15,validation_split=0.1,initial_epoch=5)

Epoch 6/15
1688/1688 [==============================] - 11s 5ms/step - loss: 0.1605 - accuracy: 0.9436 - val_loss: 0.5682 - val_accuracy: 0.8823
Epoch 7/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1589 - accuracy: 0.9476 - val_loss: 0.6228 - val_accuracy: 0.8763
Epoch 8/15
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1340 - accuracy: 0.9544 - val_loss: 0.6786 - val_accuracy: 0.8815
Epoch 9/15
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1446 - accuracy: 0.9526 - val_loss: 0.7612 - val_accuracy: 0.8732
Epoch 10/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1374 - accuracy: 0.9571 - val_loss: 0.8204 - val_accuracy: 0.8763
Epoch 11/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1242 - accuracy: 0.9601 - val_loss: 0.8677 - val_accuracy: 0.8782
Epoch 12/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1107 - accuracy: 0.9633 - val_loss: 0.9056 - val_

In [ ]:
y_predict=best_model.predict(test_images)

313/313 [==============================] - 1s 2ms/step


In [ ]:
y_predict=y_predict.astype(int)

In [ ]:
y_predict_to_array = []
for i in range(len(y_predict)):
    for j in range(10):
        if y_predict[i][j]==max(y_predict[i]):
            y_predict_to_array.append(j)

In [ ]:
y_predict_to_array = np.array(y_predict_to_array)

In [ ]:
len(y_predict_to_array)

10000

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(test_labels, y_predict_to_array)


In [ ]:
print(accuracy,'accuracy_on_test')

0.8356 accuracy_on_test
